# Calculate phase stability data and reaction affinities

## Initialize required packages and data from _01-load-exp-data notebook_

In [ ]:
from scipy import optimize
%run "core.ipynb"
analysis = load_analysis()

exp_data = analysis['exp_data']
phs_data = analysis['phs_data']
phs_mol_comp = analysis['phs_mol_comp']
phs_wt_comp = analysis['phs_wt_comp']
phs_symbols = analysis['phs_symbols'] 

## Load thermodynamic database

In [ ]:
modelDB = thermo.model.Database()

## Initialize the phase stability experiment function 

### Initiates Phase Stability Data object to retrieve valid set of reactions from SVD for each experiment 

    * input:

        exp_data: OrderedDict
            experimental conditions including T, P, fO2, and bulk starting comp

        phs_wt_comp: OrderdedDict
            Compositions of all phases in experiments indexed corresponds to experiment 
            index
        
        phase_symbols_key: Dict
            List of phase sysmbols for phases participating in reactions only
            
        modelDB: Model database object
            Thermodynamic database of choice
        
        T_units: str
            The units of temperature; options are 'C', 'K'
        
        P_units: str
            The units of pressure; options are 'bars', 'GPa'

    * output:
        
        phase_stability_data: object
            Data object containing info about each experiment including phase 
            mol compositions, endmember compositions, and valid set of reactions
            
        


In [ ]:
phase_symbols_key = LEPR_phase_symbols

In [ ]:
phase_stability_data = thermo.calibrate.PhaseStabilityData(exp_data, phs_wt_comp, phase_symbols_key, modelDB,
                                                          T_units='C', P_units='GPa')

### The number of experiments available after filtering for MELTS phases only

In [ ]:
print('There are ' + str(len(phase_stability_data._phase_stability_exps)) + ' valid experiments')

In [ ]:
print('There are ' + str(len(phase_stability_data._exp_index_invalid)) + ' experiments with phases not found in the LEPR database')

### Isolate one reaction and get look at the phase mole oxide compositions

In [ ]:
rxn_1 = phase_stability_data._phase_stability_exps[0]

In [ ]:
rxn_1._phase_mol_endmem_comp

In [ ]:
print(rxn_1._T)
print(rxn_1._P)


## Find the fraction of experiements within the ternary Py-Grs-Alm space 

### Set up endmember site occupancy and endmember stoichiometry matrices

In [ ]:
endmember_site_occ = np.array(
    [[3, 0, 0, 0, 2, 0, 0, 0, 0, 0],
     [0, 3, 0, 0, 2, 0, 0, 0, 0, 0],
     [0, 0, 3, 0, 2, 0, 0, 0, 0, 0],
     [0, 0, 0, 3, 2, 0, 0, 0, 0, 0],
     [3, 0, 0, 0, 0, 2, 0, 0, 0, 0],
     [0, 3, 0, 0, 0, 0, 1, 1, 0, 0],
     [3, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [ ]:
endmember_stoic=[]

endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MgO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'CaO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MnO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'FeO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MgO':3, 'Cr2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'CaO':3, 'TiO2':1, 'FeO':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MgO':4, 'SiO2':4}))
endmember_stoic=np.array(endmember_stoic)

In [ ]:
endmember_stoic

### Retrieve experimental garnet mol oxide data

In [ ]:
grt_mol_oxides = phs_mol_comp['Garnet']

### Get endmember compositions 

In [ ]:
endmember_comp = []
residuals = []
for imol_ox in grt_mol_oxides.values:
    try:
        iendmember_comp, iresidual = mol_oxide_to_mol_endmember(imol_ox, endmember_site_occ, 
                                                                endmember_stoic)
    except:
        iendmember_comp = None
        iresidual = None
            
            
    endmember_comp.append(iendmember_comp)
    residuals.append(iresidual)
    

### Get fraction of endmembers in the Py-Grs-Alm ternary space

In [ ]:
ternary_frac = []
for iendmember_comp in endmember_comp:
    try:
        iternary_frac = np.sum(iendmember_comp[[0,1,3]])/np.sum(iendmember_comp)
        
    except:
        iternary_frac = np.nan
        
    ternary_frac.append(iternary_frac)
    
ternary_frac = np.array(ternary_frac)    

In [ ]:
ternary_frac

# Garnet endmember space

In [ ]:
plt.figure()
plt.hist(ternary_frac[~np.isnan(ternary_frac)],70);
plt.xlabel('Fraction of garnet comp. in py-grs-alm ternary')
plt.ylabel('Number of garnet compositions')
plt.title('Experimental garnet endmember space')

# Reaction affinity calculation and visualization

### Run affinity calculation over each set of reactions for each experiment

In [ ]:
affinities = phase_stability_data.calc_equil_rxn_affinities()

In [ ]:
affinities_all = np.hstack(affinities)
affinities_clean = affinities_all[~np.isnan(affinities_all)]

plt.figure()
plt.hist(affinities_clean, 101)
plt.xlabel('rxn_affinitiy (J)')
plt.title('Histogram of reaction affinities')


## Plot fraction of ternary garnets vs. the root mean square of affinities

### Calculate rms of reaction affinities

In [ ]:
rms_affinities = []
aff_max = []
for iaff in affinities:
    rms_affinities.append(np.sqrt(np.mean(iaff**2)))
    try:
        iaff_max = np.nanmax(np.abs(iaff))
    except:
        iaff_max = np.nan
    aff_max.append(iaff_max)
    
rms_affinities=np.array(rms_affinities)
aff_max = np.array(aff_max)
    

### Plot

In [ ]:
mask = [idx not in phase_stability_data._exp_index_invalid for idx in exp_data.index]

plt.figure()
plt.plot(ternary_frac[mask],rms_affinities,'ko')
plt.xlabel('Fraction of ternary garnets (py-grs-alm)')
plt.ylabel('Affinitiy RMS')


In [ ]:
np.mean(rms_affinities>3250)